In [1]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [0]:
seed = 7
numpy.random.seed(seed)

In [3]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving iris.csv to iris.csv


In [0]:
# load dataset
dataframe = pandas.read_csv("iris.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

In [0]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

## define baseline model

In [0]:
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(8, input_dim=4, activation='relu'))
	model.add(Dense(3, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [10]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 96.00% (5.33%)


In [12]:
# define baseline model
def smaller_model():
	# create model
	model = Sequential()
	model.add(Dense(5, input_dim=4, activation='relu'))
	model.add(Dense(3, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
estimator = KerasClassifier(build_fn=smaller_model, epochs=200, batch_size=5, verbose=0)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 97.33% (3.27%)


In [15]:
def larger_model():
	# create model
    model = Sequential()
    model.add(Dense(8, input_dim=4, activation='relu'))
    model.add(Dense(8,  activation='relu'))
    model.add(Dense(6,  activation='relu'))
    model.add(Dense(3, activation='softmax'))
	# Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
estimator = KerasClassifier(build_fn=larger_model, epochs=200, batch_size=5, verbose=0)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 97.33% (4.42%)


In [16]:
# checking overfiting model
def checking_overfiting_model():
	# create model
    model = Sequential()
    model.add(Dense(8, input_dim=4, activation='relu'))
    model.add(Dense(8,  activation='relu'))
    model.add(Dense(6,  activation='relu'))
    model.add(Dense(3, activation='softmax'))
	# Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
estimator = KerasClassifier(build_fn=checking_overfiting_model, epochs=200, batch_size=5, verbose=0)

kfold = KFold(n_splits=30, shuffle=True, random_state=seed)

results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 94.67% (15.43%)


In [17]:
# checking how Much Accuracy can be acheive
def checking_accuracy_model():
	# create model
    model = Sequential()
    model.add(Dense(8, input_dim=4, activation='relu'))
    model.add(Dense(3, activation='softmax'))
	# Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
    return model
estimator = KerasClassifier(build_fn=checking_accuracy_model, epochs=100, batch_size=5, verbose=0)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 98.67% (2.67%)


In [18]:
# Rewriting using functional API model
from keras.layers import Input, Dense
from keras.models import Model

# This returns a tensor
inputs = Input(shape=(4,))

# a layer instance is callable on a tensor, and returns a tensor
x = Dense(10, activation='relu')(inputs)
x = Dense(10, activation='relu')(inputs)

predictions = Dense(3, activation='sigmoid')(x)

# This creates a model that includes
# the Input layer and two Dense layers
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history=model.fit(X, dummy_y,epochs=100, batch_size=5, verbose=False)
# print(model.summary())

history_dict = history.history
history_dict.keys()
acc_values = history_dict['acc']
print("Result: %.2f%% (%.2f%%)" % (numpy.mean(acc_values)*100, numpy.std(acc_values)*100))

Result: 86.91% (19.76%)


In [24]:
import tensorflow as tf
import keras
class MyModel(tf.keras.Model):
    
        def __init__(self):
            super(MyModel, self).__init__()
            self.dense1=tf.layers.Dense(10, activation='relu')
            self.dense2= tf.layers.Dense(8, activation='relu')

            self.dense3 =tf.layers.Dense(3, activation='sigmoid')

        def call(self, inputs):
            x = self.dense1(inputs)
            x = self.dense2(x)
            return self.dense3(x)
                        
                    
model = MyModel()
model.compile(loss='categorical_crossentropy', optimizer='adam',   metrics=['accuracy'])
history=model.fit(X,dummy_y, epochs=100,batch_size=5,verbose=False)
history_dict = history.history
history_dict.keys()
acc_values = history_dict['acc']
print("Result: %.2f%% (%.2f%%)" % (numpy.mean(acc_values)*100, numpy.std(acc_values)*100))

Result: 85.41% (17.18%)
